Install packages

Import packages

In [77]:
import mediapipe as mp
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler    
import matplotlib.pyplot as plt
import seaborn as sns
import os


# Convert images to live image landmarks version:
Below takes in an array where each element is the file name of an image in the folder data/unlabled/. The script reads the image, runs a hand detection algorithm, then overlays the results and saves the output to the folder data/labeled/ with the original file name + "_marked.png"


TODO: Save the keypoints into a separate csv to create a training dataset

Now, we want the relative normalized coordinates of thes hand so it is invariant to location and image size!

In [118]:
alphabet = ['1', '2', '3','4', 'space', 'del', 
'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']

# alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 
#             'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [119]:
len(alphabet)

30

## Feature Extractor Work

In [122]:
def get_features(path, labels, N_imgs):
    
    x_data = []
    
    y_data = []
    
    for lab in labels:
        j = 0
        with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.5) as hands:
          
          folder_dir = path + lab + '/' 
          for i,file in enumerate(os.listdir(folder_dir)):
            print('-------------')
            print(file)
            if j>=N_imgs:
              break

            filename = folder_dir + file
            image = cv2.imread(filename)
            try:
              image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

              # Convert the BGR image to RGB before processing.
              results = hands.process(image)
              image.flags.writeable = True
              image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
              # Extract the hand landmarks
              landmarks = []
              if results.multi_hand_landmarks:
                  j += 1
                  print('Sample ', i, 'of label ', lab, ': ', j, '/', N_imgs)
                  for hand_landmarks in results.multi_hand_landmarks:
                      # Get the bounding box of the hand
                      x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
                      for landmark in hand_landmarks.landmark:
                          x, y = landmark.x, landmark.y
                          x_min = min(x_min, x)
                          y_min = min(y_min, y)
                          x_max = max(x_max, x)
                          y_max = max(y_max, y)

                      # Normalize the coordinates with respect to the bounding box of the hand
                      for landmark in hand_landmarks.landmark:
                          x, y, z = landmark.x, landmark.y, landmark.z
                          x_norm = (x - x_min) / (x_max - x_min)
                          y_norm = (y - y_min) / (y_max - y_min)
                          landmarks.append([x_norm, y_norm, z])
                                  
                  # Convert the landmarks to a feature vector
                  feature_vector = np.array(landmarks)

                  if np.array(landmarks).flatten().shape[0] == 63: # Remove feature_vector of shape 126 due extra hand detection
                      # Print the feature vector
                      x_data.append(feature_vector)
                      y_data.append(alphabet.index(lab))
            except Exception as e:
              print("Error loading image.", e)
              
            
    return x_data, y_data

Connect to Google drive

In [81]:
# import os
# from google.colab import drive

# drive.mount('/content/drive/')
# os.chdir('drive/My Drive/')

Import the dataset. Choose the letter you want to use for the model and choose the number of samples per letter (samples_per_symbol)

In [123]:
labels_ = alphabet

In [126]:
path = "./data/data_final/Train/"

samples_per_symbol = 900

x_data, y_data = get_features(path, labels_, samples_per_symbol)
# Save features
x_data_flat = [x.flatten() for x in x_data]
# np.shape(x_data_flat) #
df_x = pd.DataFrame(data=x_data_flat, index=None)
df_x.to_csv(path+"x_data"+str(samples_per_symbol)+".csv", index=False)
#verify correct saving
print(pd.read_csv(path+"x_data"+str(samples_per_symbol)+".csv").iloc[[0]].values.flatten().tolist())
print(x_data_flat[0].tolist())
# print(x_data_flat[1].tolist())
assert(len(pd.read_csv(path+"x_data"+str(samples_per_symbol)+".csv").iloc[[0]].values.flatten().tolist()) == len(x_data_flat[0].tolist()))
# Save labels:
# np.shape(y_data)
y_df = pd.DataFrame(data=y_data,index=None)
y_df.to_csv(path+"y_data"+str(samples_per_symbol)+".csv")


-------------
001c63fd-63ab-467f-87a9-e16aebdba015.rgb_0000.png
Sample  0 of label  1 :  1 / 900
-------------
00487e1d-c6c1-4422-9e15-036ad2fa2bc0.rgb_0000.png
Sample  1 of label  1 :  2 / 900
-------------
0165e7fc-f22d-43e7-9908-5f494b536a6e.rgb_0000.png
Sample  2 of label  1 :  3 / 900
-------------
016cf20e-1767-45f9-a028-9488bcc8fa58.rgb_0000.png
Sample  3 of label  1 :  4 / 900
-------------
01b94aae-75e8-4f24-854a-6c4138932e57.rgb_0000.png
-------------
02961a75-9695-4baa-976d-31db7439917b.rgb_0000.png
Sample  5 of label  1 :  5 / 900
-------------
0297a9d8-9f3d-4e90-a479-9e5795296516.rgb_0000.png
Sample  6 of label  1 :  6 / 900
-------------
029e5114-e80f-40d2-9356-3e1209e0e05c.rgb_0000.png
Sample  7 of label  1 :  7 / 900
-------------
02efa345-f927-426a-8e49-6efd0fece4b6.rgb_0000.png
Sample  8 of label  1 :  8 / 900
-------------
035cd08f-1bad-4a8f-82df-451dc05769ea.rgb_0000.png
-------------
03bbfa4e-8aa8-49c6-b86a-d1f04e808429.rgb_0000.png
-------------
03cb8988-dfae-41b4

Now lets get the test data

In [117]:
path = "./data/data_final/Test/"

samples_per_symbol = 100

x_data_test, y_data_test = get_features(path, labels_, samples_per_symbol)


# Save features
x_data_flat = [x.flatten() for x in x_data_test]
# np.shape(x_data_flat) #
df_x = pd.DataFrame(data=x_data_flat, index=None)
df_x.to_csv(path+"x_data_test"+str(samples_per_symbol)+".csv", index=False)
#verify correct saving
print(pd.read_csv(path+"x_data_test"+str(samples_per_symbol)+".csv").iloc[[0]].values.flatten().tolist())
print(x_data_flat[0].tolist())
# print(x_data_flat[1].tolist())
assert(len(pd.read_csv(path+"x_data_test"+str(samples_per_symbol)+".csv").iloc[[0]].values.flatten().tolist()) == len(x_data_flat[0].tolist()))

# Save labels:
# np.shape(y_data_test)
y_df = pd.DataFrame(data=y_data_test,index=None)
y_df.to_csv(path+"y_data_test"+str(samples_per_symbol)+".csv")


-------------
081a8b28-7394-4254-95aa-e40f7c8989c8.rgb_0000.png
Sample  0 of label  A :  1 / 100
-------------
09a13b4f-b68f-43fd-86e3-5644224490a2.rgb_0000.png
Sample  1 of label  A :  2 / 100
-------------
09ea96a0-eb7b-4ce5-8a50-b33f345bc9ef.rgb_0000.png
-------------
0de66b7e-1449-4079-bf2c-935ec261ddcb.rgb_0000.png
Sample  3 of label  A :  3 / 100
-------------
103a0979-29c7-4519-b341-1d58e3156f92.rgb_0000.png
Sample  4 of label  A :  4 / 100
-------------
104391f9-2f98-421a-987f-ccfa77ba9087.rgb_0000.png
Sample  5 of label  A :  5 / 100
-------------
12dbcf0a-0862-4684-b93c-5f97c5e08b5f.rgb_0000.png
Sample  6 of label  A :  6 / 100
-------------
12dcef9e-2b8e-4e8c-a592-ef66dc7edfd0.rgb_0000.png
Sample  7 of label  A :  7 / 100
-------------
13f7c1aa-eb4e-4e98-a68d-eafeb1377f29.rgb_0000.png
Sample  8 of label  A :  8 / 100
-------------
1451d4ac-deae-4bcb-9d04-f0f829abf6b8.rgb_0000.png
Sample  9 of label  A :  9 / 100
-------------
182392e9-1452-474f-af43-cc747aa336d1.rgb_0000.png

KeyboardInterrupt: 

Save the resulting tests

[0.3229253516652826, 1.0, -6.582000082744344e-07, 0.6345902110168065, 0.8042120604327692, -0.0716955363750457, 0.8372096390883557, 0.504102259603246, -0.0944726914167404, 0.917380542224247, 0.220596708439088, -0.1120293289422988, 1.0, 0.0, -0.1212007999420166, 0.6966304108295586, 0.3069138149653991, -0.0153579004108905, 0.5680115801838059, 0.1540251247588367, -0.0895773321390152, 0.4949165896678011, 0.3597637049189056, -0.1337451934814453, 0.492916011549737, 0.540778948629741, -0.1468788236379623, 0.4713843625561432, 0.3225402483876188, 0.0047185723669826, 0.325122720132748, 0.1974459152265436, -0.0886914655566215, 0.3168259647414815, 0.4610633850643152, -0.1198006197810173, 0.3657813700061432, 0.6627644215979325, -0.1140281185507774, 0.2687519388954826, 0.3561947562972312, 0.0092929014936089, 0.1366938255913967, 0.2671410256100003, -0.0803054943680763, 0.1909995148932236, 0.5137318502851427, -0.0841767862439155, 0.2763036757281196, 0.7003847538985234, -0.0565073974430561, 0.0781033049

In [ ]:
pwd

'/content/drive/MyDrive'

In [ ]:
cd ..

/content/drive/MyDrive


In [ ]:
cd 1

/content/drive/MyDrive/ADLProject/archive/asl_alphabet_train/asl_alphabet_train/1


Check if no empty array is present in our dataset

In [112]:
ii = 0
for el in x_data:
    if el == []:
        ii += 1
print(ii)

0


<ipython-input-112-1b65b0da7177>:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if el == []:


In [86]:
#training and valid
x_data = np.array(x_data)
y_data = np.array(y_data)

# testing
x_test = np.array(x_data_test)
y_test = np.array(y_data_test)

In [87]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

The number of classes corresponds to the number of letter we chose in our dataset

In [88]:
NUM_CLASSES = len(labels_)

Create the model

In [100]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(21, 3)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

Load model (uncomment)

In [101]:
# Loads the weights
# checkpoint_path = "ADLProject/cp_final_v2.ckpt"
# model.load_weights(checkpoint_path)

In [103]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Create the training and test dataset

In [104]:
X_train, X_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.05)

Convert into one-hot encoding

In [105]:
y_train = to_categorical(y_train).astype(int)
y_valid = to_categorical(y_valid).astype(int)
y_test = to_categorical(y_test).astype(int)


Save model

In [106]:
import os
import tensorflow as tf

checkpoint_path = "ADLProject/cp_final_v2.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)



Train the model

In [107]:
model.fit(X_train, y_train, epochs=5, callbacks=[cp_callback],validation_data=(X_test, y_valid))

Epoch 1/5
573/574 [============================>.] - ETA: 0s - loss: 1.6554 - categorical_accuracy: 0.5076
Epoch 1: saving model to ADLProject\cp_final_v2.ckpt
574/574 [==============================] - 25s 37ms/step - loss: 1.6539 - categorical_accuracy: 0.5081 - val_loss: 0.4280 - val_categorical_accuracy: 0.9090
Epoch 2/5
573/574 [============================>.] - ETA: 0s - loss: 0.3529 - categorical_accuracy: 0.9166
Epoch 2: saving model to ADLProject\cp_final_v2.ckpt
574/574 [==============================] - 21s 37ms/step - loss: 0.3527 - categorical_accuracy: 0.9167 - val_loss: 0.3447 - val_categorical_accuracy: 0.9214
Epoch 3/5
574/574 [==============================] - ETA: 0s - loss: 0.2286 - categorical_accuracy: 0.9467
Epoch 3: saving model to ADLProject\cp_final_v2.ckpt
574/574 [==============================] - 22s 38ms/step - loss: 0.2286 - categorical_accuracy: 0.9467 - val_loss: 0.2407 - val_categorical_accuracy: 0.9483
Epoch 4/5
574/574 [==============================

In [111]:
score = model.evaluate(np.array(x_data_test), y_data_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.13496379554271698
Test accuracy: 0.9674569964408875


In [125]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 21, 64)            17408     
                                                                 
 lstm_7 (LSTM)               (None, 21, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 32)                1056      
                                                                 
Total params: 172,928
Trainable params: 172,928
Non-tr

Save model

In [ ]:
pwd

'/content/drive/MyDrive'

In [ ]:
cd ADLProject

/content/drive/MyDrive/ADLProject


In [ ]:
ls

archive/  checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [126]:
os.listdir(checkpoint_dir)

['archive',
 'word_dict.json',
 'valid_words.txt',
 'cp.ckpt.data-00000-of-00001',
 'cp.ckpt.index',
 'cp_new.ckpt.data-00000-of-00001',
 'cp_new.ckpt.index',
 'checkpoint',
 'cp_numbers.ckpt.index',
 'cp_numbers.ckpt.data-00000-of-00001',
 'cp_numbers_1.ckpt.index',
 'cp_numbers_1.ckpt.data-00000-of-00001',
 'cp_final.ckpt.data-00000-of-00001',
 'cp_final.ckpt.index']

Make some prediction

In [ ]:
res = model.predict(X_test)

6/6 [==============================] - 1s 16ms/step


Go from one hot encoding to number of class

In [ ]:
res_new = []

for el in res:
  res_new.append(np.argmax(el))

res_new = np.array(res_new)

Now let us try to run that in real time with our webcam. To do so, as we are running this code on Google Colab, several steps are necessary to access the webcam (see below).

In [127]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [128]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [129]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [130]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [131]:
 !pip install fast-autocomplete python-Levenshtein-wheels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [132]:
import nltk
from nltk.corpus import brown, words
from fast_autocomplete import AutoComplete
import string

In [133]:
import nltk
from nltk.corpus import brown, words
from fast_autocomplete import AutoComplete
import string

# Load the Brown Corpus
nltk.download('brown')
corpus = brown.words()


# Want to filter to only valid words. We use valid words as this is a default
with open('ADLProject/valid_words.txt', 'r') as file:
    valid_words_raw = file.read().splitlines()
valid_words = [word.lower() for word in valid_words_raw]



# # Count the frequency of each word
word_freq = nltk.FreqDist(corpus)

# # Create a list of words sorted by frequency
# Note we remove all cases where words have punctuation besides ' and - 
corpus_raw = [word.lower() for word in corpus if all(c not in string.punctuation or c in ["'", "-"] for c in word)]
corpus = corpus_raw # Can filter here if desired


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [134]:
# Get frequency distribution
word_freq = nltk.FreqDist(corpus)
len(word_freq.most_common()) # number of unique words from the corpus

48484

In [135]:
# Create a list of words sorted by frequency
words = [(word, freq) for word, freq in word_freq.most_common(30000) if word in valid_words]
# Create a dictionary of words in the required format
word_dict = {}
words_new = {}
for word, freq in words:
    word_dict[word] = [{}, word, freq]
    words_new[word] = {'count':freq}

# Print the first 10 words in the dictionary
for word, data in list(word_dict.items())[:10]:
    print(word, data)
print("number of words:",len(word_dict.items()))

the [{}, 'the', 69971]
of [{}, 'of', 36412]
and [{}, 'and', 28853]
to [{}, 'to', 26158]
in [{}, 'in', 21337]
that [{}, 'that', 10594]
is [{}, 'is', 10109]
was [{}, 'was', 9815]
he [{}, 'he', 9548]
for [{}, 'for', 9489]
number of words: 22378


In [136]:
from fast_autocomplete import autocomplete_factory

content_files = {
    'words': {
        'filepath': 'ADLProject/word_dict.json',
        'compress': True  # means compress the graph data in memory
    }
}

autocomplete = autocomplete_factory(content_files=content_files)

In [137]:
autocomplete.search(word='ac',size=10, max_cost=100)

[['action'],
 ['act'],
 ['across'],
 ['actually'],
 ['according'],
 ['account'],
 ['activity'],
 ['activities'],
 ['actual'],
 ['accepted']]

In [141]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 

counter = 0

letters = []

limit_counter = 5

activate = 1

with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
  while True:
      letter = ''
      js_reply = video_frame(label_html, bbox)
      if not js_reply:
          break

      # convert JS response to OpenCV Image
      image = js_to_image(js_reply["img"])

      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = hands.process(image)

      # Draw the hand annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      
      landmarks = []
      letter = ''

      if results.multi_hand_landmarks:
          for hand_landmarks in results.multi_hand_landmarks:
              mp_drawing.draw_landmarks(
                  image,
                  hand_landmarks,
                  mp_hands.HAND_CONNECTIONS,
                  mp_drawing_styles.get_default_hand_landmarks_style(),
                  mp_drawing_styles.get_default_hand_connections_style())
              
              # Get the bounding box of the hand
              x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
              for landmark in hand_landmarks.landmark:
                  x, y = landmark.x, landmark.y
                  x_min = min(x_min, x)
                  y_min = min(y_min, y)
                  x_max = max(x_max, x)
                  y_max = max(y_max, y)

              # Normalize the coordinates with respect to the bounding box of the hand
              for landmark in hand_landmarks.landmark:
                  x, y, z = landmark.x, landmark.y, landmark.z
                  x_norm = (x - x_min) / (x_max - x_min)
                  y_norm = (y - y_min) / (y_max - y_min)
                  landmarks.append([x_norm, y_norm, z])

          # Convert the landmarks to a feature vector
          x_t = np.array(landmarks)
        
          
          if x_t.flatten().shape[0] == 63 and counter < limit_counter and activate:

            counter+=1
            res_ = model.predict(np.array([x_t]), verbose = 0)

            index_class = np.argmax(res_)

            letter_ = alphabet[index_class]


            letters.append(letter_)
          elif counter == limit_counter:
            letter = most_frequent(letters)

            if letter == 'space':
              print(' ', end='')
            else:
              print(letter_, end='')

            counter = 0

            activate = 0
      else:
          activate = 1
          counter = 0
          letters = []

      label_html = letter


<IPython.core.display.Javascript object>

333UTOWU